In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Conv2D, MaxPooling2D,
    Flatten, Dense, Dropout
)
from tensorflow.keras.utils import to_categorical

In [ ]:
sns.set(style="darkgrid")
np.random.seed(42)
tf.random.set_seed(42)

LOAD DATASET

In [ ]:
digits = load_digits()

X = digits.images     
y = digits.target     
print("Original image shape:", X.shape)

VISUALIZE SAMPLE IMAGES

In [ ]:
plt.figure(figsize=(10, 4))
for i in range(10):
    plt.subplot(2, 5, i + 1)
    plt.imshow(X[i], cmap="gray")
    plt.title(f"Digit: {y[i]}")
    plt.axis("off")
plt.suptitle("Sample Digits (8x8 Images)")
plt.show()

DATA PREPROCESSING FOR CNN

In [ ]:
# Normalize
X = X / 16.0

# Add channel dimension  (N, 8, 8, 1)
X = X.reshape(-1, 8, 8, 1)

# One-hot
y_cat = to_categorical(y, num_classes=10)

# Train-test split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y_cat,
    test_size=0.2,
    random_state=42,
    stratify=y
)
print("CNN Input shape:", X_train.shape)

BASELINE CNN MODEL

In [ ]:
baseline_cnn = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation="relu", input_shape=(8, 8, 1)),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    Dense(64, activation="relu"),
    Dense(10, activation="softmax")
])

In [ ]:
baseline_cnn.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

baseline_cnn.summary()

TRAIN BASELINE CNN

In [ ]:
history_base = baseline_cnn.fit(
    X_train, y_train,
    epochs=30,
    batch_size=32,
    validation_split=0.2,
    verbose=1
)

IMPROVED CNN MODEL

In [ ]:
improved_cnn = Sequential([
    Conv2D(32, (3, 3), activation="relu", padding="same", input_shape=(8, 8, 1)),
    Conv2D(64, (3, 3), activation="relu", padding="same"),
    MaxPooling2D((2, 2)),
    Dropout(0.25),

    Flatten(),
    Dense(128, activation="relu"),
    Dropout(0.4),
    Dense(10, activation="softmax")
])

In [ ]:
improved_cnn.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

improved_cnn.summary()

TRAIN IMPROVED CNN

In [ ]:
history_improved = improved_cnn.fit(
    X_train, y_train,
    epochs=50,
    batch_size=32,
    validation_split=0.2,
    verbose=1
)

EVALUATE CNN ON TEST DATA

In [ ]:
test_loss, test_acc = improved_cnn.evaluate(X_test, y_test)
print("\nImproved CNN Test Accuracy:", test_acc)

CONFUSION MATRIX

In [ ]:
y_pred = np.argmax(improved_cnn.predict(X_test), axis=1)
y_true = np.argmax(y_test, axis=1)

cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(7, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.title("CNN Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()

print(classification_report(y_true, y_pred))